In [50]:
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [51]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')
print("Model loaded")

Model loaded


In [52]:
all_movies = pd.read_csv("movies.csv")["description"]
all_movies

0      Пострадав в результате несчастного случая, бог...
1      Один ушлый американец ещё со студенческих лет ...
2      1987 год. Джордан Белфорт становится брокером ...
3      Демобилизовавшись, Данила Багров вернулся в ро...
4      Грузовики лос-анджелесской инкассаторской комп...
                             ...                        
995    Ковбой Вуди и другие игрушки мальчика по имени...
996    Лабиринт пройден, но Томасу, Терезе, Минхо и п...
997    Любовная связь с женатым мужчиной – это, конеч...
998    В Лос-Анджелесе злодеи похищают малолетнюю доч...
999    Торакальный хирург Женя Беляева за месяц должн...
Name: description, Length: 1000, dtype: object

In [53]:
all_movies_names = pd.read_csv("movies.csv")["name"]
all_movies_names

0                                       1+1
1                               Джентльмены
2                         Волк с Уолл-стрит
3                                      Брат
4                         Гнев человеческий
                       ...                 
995                       История игрушек 2
996    Бегущий в лабиринте: Испытание огнём
997                          Другая женщина
998                                 Час пик
999                                   Вызов
Name: name, Length: 1000, dtype: object

In [54]:
offset = 0
batch_size = 2

while offset < 1000:
    movies = all_movies[offset : offset + batch_size]
    movies = "query: " + movies
    movies = [*movies]
    print(offset)
    
    # Tokenize the input texts:
    batch_dict = tokenizer(movies, max_length=512, padding=True, truncation=True, return_tensors='pt')
    
    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    
    # Normalize embeddings:
    embeddings : Tensor = F.normalize(embeddings, p=2, dim=1)

    np_embeddings = embeddings.detach().numpy()
    df = pd.DataFrame(np_embeddings)
    df.index = all_movies_names[offset : offset + batch_size]
    if offset == 0:
        df.to_csv("embeddings.csv", mode="a", header=True)
    else:
        df.to_csv("embeddings.csv", mode="a", header=False)
    
    offset += batch_size

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
326
328
330
332
334
336
338
340
342
344
346
348
350
352
354
356
358
360
362
364
366
368
370
372
374
376
378
380
382
384
386
388
390
392
394
396
398
400
402
404
406
408
410
412
414
416
418
420
422
424
426
428
430
432
434
436
438
440
442
444
446
448
450
452
454
456
458
460
462
464
466
468
470
472
474
476
478
480
482
484
486
488
490
492
494
496
498
500
502
504
506
508
510
512
514
516
518
520
522
524
526

In [55]:
df = pd.read_csv("embeddings.csv").to_numpy()
movies_names = df[:, 0]
embeddings = df[: , 1:]
scores = (embeddings @ embeddings.T) * 100

In [107]:
df_movies_x_movies = pd.DataFrame(scores.tolist(), columns=movies_names, index=movies_names)
print(df_movies_x_movies)

                                            1+1  Джентльмены  \
1+1                                   99.999993    81.704281   
Джентльмены                           81.704281    99.999999   
Волк с Уолл-стрит                     80.846187    85.542187   
Брат                                  83.183323    81.130072   
Гнев человеческий                     80.647662    81.678488   
...                                         ...          ...   
История игрушек 2                     80.368407    80.955034   
Бегущий в лабиринте: Испытание огнём  80.194607    78.666079   
Другая женщина                        79.672268    80.872690   
Час пик                               82.254772    84.142275   
Вызов                                 81.243327    77.161918   

                                      Волк с Уолл-стрит       Брат  \
1+1                                           80.846187  83.183323   
Джентльмены                                   85.542187  81.130072   
Волк с Уолл-стрит    

In [108]:
df_movies_x_movies.to_csv('movies_similarities.csv', index=True, header=True)

In [137]:
movies_dict = {}
movies_str = ""
counter = 1

for movie in df_movies_x_movies.index:
    movies_dict[counter] = movie
    movies_str = movies_str + str(counter) + ": " + movie + "\n"
    counter += 1

f = open("movies.txt", "w")
f.write(movies_str)
f.close()

In [141]:
print(movies_str)

1: 1+1
2: Джентльмены
3: Волк с Уолл-стрит
4: Брат
5: Гнев человеческий
6: Один дома
7: Аватар
8: Побег из Шоушенка
9: Зверополис
10: Достать ножи
11: Игра престолов
12: Иван Васильевич меняет профессию
13: Начало
14: Бойцовский клуб
15: Зеленая миля
16: Гарри Поттер и философский камень
17: Брат 2
18: Пираты Карибского моря: Проклятие Черной жемчужины
19: Зеленая книга
20: Интерстеллар
21: Любовь и голуби
22: Остров проклятых
23: Операция «Ы» и другие приключения Шурика
24: Триггер
25: Рик и Морти
26: Форрест Гамп
27: Веном
28: Титаник
29: Мстители: Война бесконечности
30: Король и Шут
31: Монастырь
32: Мстители: Финал
33: Легенда
34: Мажор
35: Ну, погоди!
36: Шрэк
37: Криминальное чтиво
38: Области тьмы
39: Рататуй
40: Поймай меня, если сможешь
41: Стражи Галактики
42: Джентльмены удачи
43: Переводчик
44: Дюна
45: Доктор Стрэндж
46: Матрица
47: Темный рыцарь
48: Бриллиантовая рука
49: Мстители
50: Тайна Коко
51: Джанго освобожденный
52: Леон
53: Кавказская пленница, или Новые приключ

In [146]:
movie_to_search = int(input("Введите номер желаемого фильма:\n"))
movie_name = movies_dict[movie_to_search]
print(df_movies_x_movies[movie_name].sort_values(ascending=True))

Губка Боб квадратные штаны      76.191375
Матрица: Революция              77.462005
Бёрдмэн                         77.715507
Молодой Папа                    77.787497
Отличница лёгкого поведения     78.038499
                                  ...    
Бригада                         87.118258
Мажор                           87.264377
Холоп                           87.290696
Патруль                         87.307323
Каникулы строгого режима       100.000000
Name: Каникулы строгого режима, Length: 1000, dtype: float64
